In [1]:
%serialconnect

Connecting to --port=/dev/ttyUSB3 --baud=115200 
Ready.


In [ ]:
# Apparently there is a sensor fusion DMP digital motion processor
# that should be able to process on the device 
# https://github.com/micropython-IMU/micropython-mpu9150/issues/6


In [ ]:
from machine import I2C, Pin
rst = Pin(16, Pin.OUT)
rst.value(1)

i2c = I2C(scl=Pin(15, Pin.OUT, Pin.PULL_UP), sda=Pin(4, Pin.OUT, Pin.PULL_UP), freq=450000)
ads = i2c.scan()
print([hex(s)  for s in ads])
if 0x68 in ads:
    print("We have an MPU6050")


['0x68']
We have an MPU6050


In [ ]:
i2c.writeto_mem(0x68, 0x6B, b"\x00")  # disable sleep mode and select clock source as the XGyro


In [ ]:
import ustruct
bdata = i2c.readfrom_mem(0x68, 0x3B, 14)
data = ustruct.unpack('>hhhhhhh', bdata)
accel = [x/(65536//2//2) for x in data[0:3]]
gyros = [x/(65536//250//2) for x in data[4:7]]
print(accel, gyros)


[0.9226074, -0.02099609, 0.4555664] [-2.755725, 0.4656488, -1.045802]


In [ ]:
import time

print(hex(i2c.readfrom_mem(0x68, 0x75, 1)[0]))  # who am I (the I2C address)

# look at MPU6050 register map datasheet
i2c.writeto_mem(0x68, 0x6B, b"\x80")  # reset device
time.sleep_ms(100)
i2c.writeto_mem(0x68, 0x68, b"\x07")  # reset gyro, accel, temp
time.sleep_ms(100)


0x68


In [ ]:
i2c.writeto_mem(0x68, 0x6B, b"\x01")  # disable sleep mode and select clock source as the XGyro
i2c.writeto_mem(0x68, 0x6C, b"\x00")  # enable all sensors (standby=0)
i2c.writeto_mem(0x68, 0x1A, b"\x01")  # Enable digital low pass filter (DLPF)
i2c.writeto_mem(0x68, 0x19, b"\x20")  # sample rate is 33Hz=1kH/(1+rate) (1kH because DLPF enabled)

accel_range = 2  # g m/s
v = i2c.readfrom_mem(0x68, 0x1C, 1)[0]
v = (v&~0x18 | 0x00)
i2c.writeto_mem(0x68, 0x1C, bytes([v]))

gyro_range = 250  # deg/sec
v = i2c.readfrom_mem(0x68, 0x1B, 1)[0]
v = (v&~0x18 | 0x00)
i2c.writeto_mem(0x68, 0x1B, bytes([v]))

import ustruct, time
bdata = bytearray(14)

In [ ]:
for i in range(10):
    i2c.readfrom_mem_into(0x68, 0x3B, bdata)
    data = ustruct.unpack('>hhhhhhh', bdata)
    accel = [x/(65536//2//2) for x in data[0:3]]
    gyros = [x/(65536//250//2) for x in data[4:7]]
    print("accel:", accel, "gyro:", gyros)
    time.sleep(1)

accel: [0.9265137, -0.01477051, 0.4494629] gyro: [-2.580153, 0.5954198, -0.8015267]
accel: [0.5335693, -0.3244629, 0.5107422] gyro: [19.49618, -37.51909, 25.07634]
accel: [0.9155273, -0.02661133, 0.4489746] gyro: [-2.458015, 0.7328244, -1.045802]
accel: [-0.2254639, -0.1268311, 1.096069] gyro: [12.03053, 250.1298, 1.839695]
accel: [-0.4547119, 0.001831055, 0.8961182] gyro: [-3.946565, -1.526718, -0.8015267]
accel: [-0.4558105, -0.006835938, 0.8970947] gyro: [-3.022901, 0.2213741, -0.8473283]
accel: [0.8946533, -0.09216309, 0.5523682] gyro: [-36.93893, -250.1374, -11.57252]
accel: [0.3571777, 0.1170654, -1.069092] gyro: [-37.58779, -36.9542, -2.916031]
accel: [0.5683594, 0.6875, -0.2215576] gyro: [-149.4275, 190.7252, 13.81679]
accel: [0.9960938, 0.3682861, -0.2001953] gyro: [-5.145038, 20.35878, 206.0076]
.